In [ ]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
   fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
   while True:
       match = NOT_WHITESPACE.search(document, pos)
       if not match:
           return
       pos = match.start()

       try:
           obj, pos = decoder.raw_decode(document, pos)
       except JSONDecodeError:
           # do something sensible if there's some error
            print("YOU ARE WRONG")
           pass
       yield obj

In [ ]:
print(list(decode_stacked(fc)))